In [1]:
#Practice with some ConvNets
#This ipython also assumes you have decent knowledge of ConvNets. This article should be good enough for now: https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53

In [2]:
#Let's import libraries 
import tensorflow as tf
import numpy as np
%matplotlib inline 
import matplotlib.pyplot as plt

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/

In [3]:
#We'll start with MNIST first
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
"""(x_train, y_train), (x_test, y_test) tf.keras.datasets.cifar10.load_data()
In case you want to try CIFAR (NOTE: It would be good to check shape of Cifar and properly adjust since it is RGB)
"""

'(x_train, y_train), (x_test, y_test) tf.keras.datasets.cifar10.load_data()\nIn case you want to try CIFAR (NOTE: It would be good to check shape of Cifar and properly adjust since it is RGB)\n'

In [4]:
#Something to note is that CNN layers taken in channels as input as well. Since MNIST is grayscale, we just need to add singular channel dimension
x_train = np.expand_dims(x_train, axis=3)
x_test = np.expand_dims(x_test, axis=3)
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [5]:
#One-hot encoding
numpy_zero = np.zeros((y_train.shape[0], 10))
numpy_zero[np.arange(y_train.shape[0]), y_train] = 1
y_train = numpy_zero 
numpy_zero2 = np.zeros((y_test.shape[0], 10))
numpy_zero2[np.arange(y_test.shape[0]), y_test] = 1
y_test = numpy_zero2

In [6]:
#Don't forget to scale values
x_train = x_train / 255.0
x_test = x_test / 255.0 

In [7]:
#Ok so far we've just been doing the same things as our previous notebook. Let's actually get into the meat of ConvNets.
#Again, it would be good if you understand just somewhat intuitively what ConvNets do each layer before coding

In [8]:
#define our convnet model
def ConvNet(inputs):
    #encoding
    c1 = tf.layers.conv2d(inputs=inputs, filters=16, kernel_size=(3, 3), padding='same', activation=tf.nn.relu)
    p1 = tf.layers.max_pooling2d(inputs=c1,pool_size=(2,2),strides=(2,2),padding='valid')
    
    c2 = tf.layers.conv2d(inputs=p1, filters=8, kernel_size=(3, 3), padding='same', activation=tf.nn.relu)
    p2 = tf.layers.max_pooling2d(inputs=c2,pool_size=(2,2),strides=(2,2),padding='valid')
    
    d3 = tf.layers.dense(inputs=tf.layers.flatten(p2), units=32, activation=tf.nn.relu)
    #d3 - representation
    #starting decode
    d4 = tf.layers.dense(inputs=d3, units=7*7*8, activation=tf.nn.relu)
    
    tc5 = tf.layers.conv2d_transpose(inputs=tf.reshape(d4,(-1,7,7,8)), strides=(2,2), filters=16, kernel_size=(3,3), activation=tf.nn.relu, padding='same')
    
    tc6 = tf.layers.conv2d_transpose(inputs=tc5, filters=1, strides=(2,2), kernel_size=(3,3), activation=tf.nn.sigmoid, padding='same')
    
    return tc6

In [9]:
#Now let's walk through the same model building as last time
inputs=tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
#We define our input differently in this case

labels = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])

outputs = ConvNet(inputs)

loss = tf.reduce_mean(tf.pow(outputs - inputs, 2))

trainer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

#Let's set some hyperparamaters for batch size and epochs
batch_size = 128
num_epochs = 10
    

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use `tf.keras.layers.Conv2DTranspose` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
init=tf.global_variables_initializer()
local_init = tf.local_variables_initializer()
with tf.Session() as sess:
    #each call to sess.run() runs the approriate input within our tf session
    sess.run(init)
    sess.run(local_init)
    print("Starting to run session...")
    for epoch in range(num_epochs): #iterate through each epoch
        for i in range(x_train.shape[0]//batch_size): #iterate through the number of batches we can make from our samples
            batch_images = x_train[i*batch_size:(i+1)*batch_size]
            #We have a batch of training images and labels. 
            #Now we need to run the model by inputing these into the place holders we made
            _ = sess.run([trainer], feed_dict={inputs: batch_images, labels: batch_images})
        #Checking our accuracy each epoch...
        l = sess.run(
            loss, feed_dict={inputs: x_test})
        print(l)
        
    pred = sess.run(
            outputs, feed_dict={inputs: x_test})
    f, a = plt.subplots(2, 10, figsize=(10, 2))
    for i in range(10):
        a[0][i].imshow(np.reshape(x_test[i], (28, 28)))
        a[1][i].imshow(np.reshape(pred[i], (28, 28)))
    plt.show()
    plt.draw()
        # Compare original images with their reconstructions


Starting to run session...
0.047878623
0.024642358
0.019301577
0.016180357
0.014383041
0.01336227
0.012767703
